In [1]:
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.impute import KNNImputer
from joblib import parallel_backend
from sklearn.model_selection import train_test_split, RepeatedKFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import PowerTransformer, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from hummingbird.ml import convert, load

C:\Users\Pabma\AppData\Local\Temp\ipykernel_18444\3539060139.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
ventas_oil = pd.read_csv('ventas_oil_imputed.csv')
ventas_elec = pd.read_csv('ventas_elec_imputed.csv')

In [3]:
ventas_oil.dtypes


price_amount                     float64
makeid                           float64
manufacturerprice                float64
km                               float64
vehicleyear                      float64
etiqueta_type_id                 float64
provinceid                       float64
horsepower                       float64
maxspeed                         float64
acceleration                     float64
combustible_type_id              float64
body_type_id                     float64
transmision_type_id              float64
doors                            float64
seatingcapacity                  float64
colores_type_id                  float64
dimensionsinmillimeterswidth     float64
dimensionsinmillimetersheight    float64
dimensionsinmillimeterslength    float64
weight                           float64
tankcapacityinliters             float64
trunkcapacityinliters            float64
consumptionurban                 float64
consumptionmixed                 float64
consumptionextra

In [4]:
ventas_oil['makeid'] = ventas_oil['makeid'].astype('category')
ventas_oil['etiqueta_type_id'] = ventas_oil['etiqueta_type_id'].astype('category')
ventas_oil['provinceid'] = ventas_oil['provinceid'].astype('category')
ventas_oil['transmision_type_id'] = ventas_oil['transmision_type_id'].astype('category')
ventas_oil['body_type_id'] = ventas_oil['body_type_id'].astype('category')
ventas_oil['combustible_type_id'] = ventas_oil['combustible_type_id'].astype('category')
ventas_oil['colores_type_id'] = ventas_oil['colores_type_id'].astype('category')
ventas_oil['lPrice'] = np.log10(ventas_oil['price_amount'])

In [5]:
ventas_oil.dtypes

price_amount                      float64
makeid                           category
manufacturerprice                 float64
km                                float64
vehicleyear                       float64
etiqueta_type_id                 category
provinceid                       category
horsepower                        float64
maxspeed                          float64
acceleration                      float64
combustible_type_id              category
body_type_id                     category
transmision_type_id              category
doors                             float64
seatingcapacity                   float64
colores_type_id                  category
dimensionsinmillimeterswidth      float64
dimensionsinmillimetersheight     float64
dimensionsinmillimeterslength     float64
weight                            float64
tankcapacityinliters              float64
trunkcapacityinliters             float64
consumptionurban                  float64
consumptionmixed                  

In [6]:
ventas_elec.dtypes

price_amount                     float64
makeid                           float64
manufacturerprice                float64
km                               float64
vehicleyear                      float64
etiqueta_type_id                 float64
provinceid                       float64
horsepower                       float64
maxspeed                         float64
acceleration                     float64
combustible_type_id              float64
body_type_id                     float64
transmision_type_id              float64
doors                            float64
seatingcapacity                  float64
colores_type_id                  float64
dimensionsinmillimeterswidth     float64
dimensionsinmillimetersheight    float64
dimensionsinmillimeterslength    float64
weight                           float64
tankcapacityinliters             float64
trunkcapacityinliters            float64
consumptionurban                 float64
consumptionmixed                 float64
consumptionextra

In [7]:
ventas_elec['makeid'] = ventas_elec['makeid'].astype('category')
ventas_elec['etiqueta_type_id'] = ventas_elec['etiqueta_type_id'].astype('category')
ventas_elec['provinceid'] = ventas_elec['provinceid'].astype('category')
ventas_elec['transmision_type_id'] = ventas_elec['transmision_type_id'].astype('category')
ventas_elec['body_type_id'] = ventas_elec['body_type_id'].astype('category')
ventas_elec['combustible_type_id'] = ventas_elec['combustible_type_id'].astype('category')
ventas_elec['colores_type_id'] = ventas_elec['colores_type_id'].astype('category')
ventas_elec['lPrice'] = np.log10(ventas_elec['price_amount'])

In [8]:
ventas_elec.dtypes

price_amount                      float64
makeid                           category
manufacturerprice                 float64
km                                float64
vehicleyear                       float64
etiqueta_type_id                 category
provinceid                       category
horsepower                        float64
maxspeed                          float64
acceleration                      float64
combustible_type_id              category
body_type_id                     category
transmision_type_id              category
doors                             float64
seatingcapacity                   float64
colores_type_id                  category
dimensionsinmillimeterswidth      float64
dimensionsinmillimetersheight     float64
dimensionsinmillimeterslength     float64
weight                            float64
tankcapacityinliters              float64
trunkcapacityinliters             float64
consumptionurban                  float64
consumptionmixed                  

In [12]:
# Separamos en training y test 50% para training y 50% para test
ventas_oil_X = ventas_oil.drop(columns=['price_amount', 'lPrice'])
ventas_oil_y = ventas_oil['lPrice']
ventas_oil_X_train, ventas_oil_X_test, ventas_oil_y_train, ventas_oil_y_test = train_test_split(ventas_oil_X, ventas_oil_y, test_size=0.45, random_state=123)

# Separamos en training y test 75% para training y 25% para test
ventas_elec_X = ventas_elec.drop(columns=['price_amount', 'lPrice'])
ventas_elec_y = ventas_elec['lPrice']
ventas_elec_X_train, ventas_elec_X_test, ventas_elec_y_train, ventas_elec_y_test = train_test_split(ventas_elec_X, ventas_elec_y, test_size=0.25, random_state=123)

# El 55% de los datos de training se dividen en un 80% para training y 20% para validación
ventas_oil_X_train_split, ventas_oil_X_val, ventas_oil_y_train_split, ventas_oil_y_val = train_test_split(ventas_oil_X_train, ventas_oil_y_train, test_size=0.2, random_state=456)

# El 80% de los datos de training se dividen en un 80% para training y 20% para validación
ventas_elec_X_train_split, ventas_elec_X_val, ventas_elec_y_train_split, ventas_elec_y_val = train_test_split(ventas_elec_X_train, ventas_elec_y_train, test_size=0.2, random_state=456)


In [24]:
# Crear objetos validacion cruzada para trial, tuning y assessment
rk_trial = RepeatedKFold(n_splits=5, n_repeats=1, random_state=123)
rk_tuning = RepeatedKFold(n_splits=5, n_repeats=1, random_state=456)
rk_assessment = RepeatedKFold(n_splits=5, n_repeats=1, random_state=789)

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def mae(y_true, y_pred):
    return mean_absolute_error(y_true, y_pred)

def rsq(y_true, y_pred):
    return r2_score(y_true, y_pred)

numeric_features_oil = ventas_oil_X_train.select_dtypes(include=['int', 'float']).columns
preprocessor_oil = ColumnTransformer(
    transformers=[
        ('yeo_johnson', PowerTransformer(method='yeo-johnson'), ['km']),
        ('scaling', StandardScaler(), numeric_features_oil)
    ])

numeric_features_elec = ventas_elec_X_train.select_dtypes(include=['int', 'float']).columns
preprocessor_elec = ColumnTransformer(
    transformers=[
        ('yeo_johnson', PowerTransformer(method='yeo-johnson'), ['km']),
        ('scaling', StandardScaler(), numeric_features_elec)
    ])

rf_model_trial = RandomForestRegressor(n_estimators=1000, min_samples_leaf=15)
rf_pipeline_trial_oil = Pipeline(steps=[('preprocessor', preprocessor_oil), ('model', rf_model_trial)])
rf_pipeline_trial_elec = Pipeline(steps=[('preprocessor', preprocessor_elec), ('model', rf_model_trial)])

In [20]:
resultados_oil = {}
index = 0
for train_index, test_index in rk_trial.split(ventas_oil_X_train):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_trial_train, X_trial_test = ventas_oil_X_train.iloc[train_index], ventas_oil_X_train.iloc[test_index]
    y_trial_train, y_trial_test = ventas_oil_y_train.iloc[train_index], ventas_oil_y_train.iloc[test_index]

    with parallel_backend('threading', n_jobs=8):
        rf_pipeline_trial_oil.fit(X_trial_train, y_trial_train)
    
    y_pred = rf_pipeline_trial_oil.predict(X_trial_test)
    rmse_value = rmse(y_trial_test, y_pred)
    mae_value = mae(y_trial_test, y_pred)
    rsq_value = rsq(y_trial_test, y_pred)
    metricS = {'RMSE': rmse_value, 'MAE': mae_value, 'R^2': rsq_value}
    index += 1
    resultados_oil[f'Iteración_{index}'] = {'RMSE': rmse_value,
                                      'MAE': mae_value,
                                      'R^2': rsq_value}
    

TRAIN: [    0     1     3 ... 85754 85756 85757] TEST: [    2     8    10 ... 85747 85755 85758]
TRAIN: [    0     2     3 ... 85755 85757 85758] TEST: [    1     5     7 ... 85751 85754 85756]
TRAIN: [    0     1     2 ... 85756 85757 85758] TEST: [   16    19    21 ... 85738 85739 85740]
TRAIN: [    0     1     2 ... 85755 85756 85758] TEST: [    3    14    28 ... 85746 85750 85757]
TRAIN: [    1     2     3 ... 85756 85757 85758] TEST: [    0     4     6 ... 85749 85752 85753]


In [25]:
resultados_elec = {}
index = 0
for train_index, test_index in rk_trial.split(ventas_elec_X_train):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_trial_train, X_trial_test = ventas_elec_X_train.iloc[train_index], ventas_elec_X_train.iloc[test_index]
    y_trial_train, y_trial_test = ventas_elec_y_train.iloc[train_index], ventas_elec_y_train.iloc[test_index]

    with parallel_backend('threading', n_jobs=8):
        rf_pipeline_trial_elec.fit(X_trial_train, y_trial_train)
    
    y_pred = rf_pipeline_trial_elec.predict(X_trial_test)
    rmse_value = rmse(y_trial_test, y_pred)
    mae_value = mae(y_trial_test, y_pred)
    rsq_value = rsq(y_trial_test, y_pred)
    metricS = {'RMSE': rmse_value, 'MAE': mae_value, 'R^2': rsq_value}
    index += 1
    resultados_elec[f'Iteración_{index}'] = {'RMSE': rmse_value,
                                      'MAE': mae_value,
                                      'R^2': rsq_value}

TRAIN: [   0    1    2 ... 7320 7321 7322] TEST: [   4    6    9 ... 7310 7312 7313]
TRAIN: [   0    1    2 ... 7320 7321 7322] TEST: [  10   12   15 ... 7299 7301 7318]
TRAIN: [   0    1    2 ... 7318 7319 7320] TEST: [   5   18   19 ... 7316 7321 7322]
TRAIN: [   2    3    4 ... 7320 7321 7322] TEST: [   0    1    7 ... 7306 7317 7319]
TRAIN: [   0    1    4 ... 7319 7321 7322] TEST: [   2    3    8 ... 7307 7315 7320]


In [22]:
for key, value in resultados_oil.items():
    print(key, ": ", value)
    print()

Iteración_1 :  {'RMSE': 0.09827536319095209, 'MAE': 0.06250605992130741, 'R^2': 0.9120503778350576}

Iteración_2 :  {'RMSE': 0.09495735431835219, 'MAE': 0.061333026888323684, 'R^2': 0.9188287295747345}

Iteración_3 :  {'RMSE': 0.0940589907316913, 'MAE': 0.06105159432854758, 'R^2': 0.9179003405088485}

Iteración_4 :  {'RMSE': 0.09558931726580508, 'MAE': 0.06171662988295184, 'R^2': 0.9161772374093423}

Iteración_5 :  {'RMSE': 0.09623033794511875, 'MAE': 0.06177299379084846, 'R^2': 0.9156090440832573}



In [26]:
for key, value in resultados_elec.items():
    print(key, ": ", value)
    print()

Iteración_1 :  {'RMSE': 0.05415098137248, 'MAE': 0.03453586986846301, 'R^2': 0.931434943156957}

Iteración_2 :  {'RMSE': 0.050387462553125414, 'MAE': 0.032523631738270026, 'R^2': 0.9419842841588791}

Iteración_3 :  {'RMSE': 0.051390339702198855, 'MAE': 0.032918490414070654, 'R^2': 0.9335481077454139}

Iteración_4 :  {'RMSE': 0.048354910987534375, 'MAE': 0.03262291998184615, 'R^2': 0.9422105314077719}

Iteración_5 :  {'RMSE': 0.051637580766156016, 'MAE': 0.031682135026915664, 'R^2': 0.9341847101370689}

